In [5]:
from transformers import pipeline, AutoModelForTokenClassification, BertTokenizerFast
from absa import ABSAModel

ner_model_path = "/home/camp5/karayel/deneme_model/bert_model"

finetuned_model = AutoModelForTokenClassification.from_pretrained(ner_model_path)
tokenizer = BertTokenizerFast.from_pretrained(ner_model_path)

def response_dondur(sentence, finetuned_model=finetuned_model, tokenizer=tokenizer):

    sentence=sentence.lower()
    
    ner = pipeline("ner", model = finetuned_model, tokenizer = tokenizer, device=0)
    result = ner(sentence)

    entities = [i for i in result if i['entity']=='B-ORG']

    modelABSA = ABSAModel(tokenizer, adapter=False)

    absa_model_path = "/karayel/model_ABSA_adapter/model_lr1e-05_epochs0_batch8.pkl"

    json_1 = {}
    json_1['entity_list']=[i['word'] for i in entities]

    temp = []
    
    for count, i in enumerate(entities):
    
        x, y, z = modelABSA.predict(sentence, i['word'], load_model=absa_model_path)
    
        if y==1:
            temp.append({'entity':i['word'], 'sentiment':'olumlu'})
        elif y==0:
            temp.append({'entity':i['word'], 'sentiment':'nötr'})
        else:
            temp.append({'entity':i['word'], 'sentiment':'olumsuz'})

        if count==len(entities)-1:
            json_1['result'] = temp
            
    return json_1

In [6]:
response_dondur('Turkcell ne kadar güzel bir firma böyle')

{'entity_list': ['turkcell'],
 'result': [{'entity': 'turkcell', 'sentiment': 'olumsuz'}]}

In [236]:
import json
import re

def encode(data):
    # Metni kelimelere ayır (Türkçe karakterleri koruyarak)
    words = re.findall(r'\b\w+\b', data['sentence'])
    words = [i.lower() for i in words]
    # Entity ve duygu durumlarını dönüştürmek için yardımcı sözlükler
    entity_to_id = {entity: idx + 1 for idx, entity in enumerate(data['entity_list'])}
    sentiment_map = {'negative': 0, 'neutral': 1, 'positive': 2}
    
    # Entity ve duygu durumlarını belirlemek için listeler
    entities = []
    sentiments = []
    
    # Entity listesi ve sonuçları daha hızlı erişmek için dictionary
    entity_dict = {result['entity']: result['sentiment'] for result in data['results']}
    
    # Entity işaretleme
    i = 0
    while i < len(words):
        word = words[i]
        # Çok kelimeli entity kontrolü
        matched = False
        for j in range(len(data['entity_list'])):
            entity = data['entity_list'][j]
            entity_parts = entity.split()
            if word == entity_parts[0] and ' '.join(words[i:i+len(entity_parts)]) == entity:
                # İlk kelimeyi 1, diğer kelimeleri 2 olarak encode et
                entities.extend([1] + [2] * (len(entity_parts) - 1))
                # Duygu durumunu ekle
                sentiment = sentiment_map.get(entity_dict.get(entity, 'neutral'), 1)
                sentiments.extend([sentiment] * len(entity_parts))
                i += len(entity_parts)
                matched = True
                break
            elif word.count(entity_parts[0])>0:
                entities.extend([1] + [2] * (len(entity_parts) - 1))
                # Duygu durumunu ekle
                sentiment = sentiment_map.get(entity_dict.get(entity, 'neutral'), 1)
                sentiments.extend([sentiment] * len(entity_parts))
                i += len(entity_parts)
                matched = True
                break
                
        if not matched:
            # Kelime entity değilse
            entities.append(0)
            sentiments.append(-1)
            i += 1
    
    # Sonuçları formatla
   # result = [
   #     json.dumps(words, ensure_ascii=False),
  #      json.dumps(entities, ensure_ascii=False),
  #      json.dumps(sentiments, ensure_ascii=False)
#    ]

    result = [
        words, entities, sentiments
    ]

    return pd.Series([words, entities, sentiments], index=["Tokens","Tags","Polarities"])